In [ ]:
#!pip install msgpack
#!pip install ipython
#!pip install pyspark==2.3.0
#!python -m pip install --upgrade pip
#!pip install pyLDAvis

In [1]:
import os
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[8] pyspark-shell"
os.environ["JAVA_HOME"] = "C:\Java\jdk1.8.0_172"
#USATO PER RISOLVEREE PROBLEMA JAVA GATEWAY ..... (ho anche installato java e inserito manualmente java home tra le variabili ambiente)

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA, BisectingKMeans
from pyspark.sql.functions import monotonically_increasing_id
import re
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.stem.snowball import SnowballStemmer
import nltk
from nltk.corpus import stopwords
import pyspark
import string
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.mllib.util import MLUtils
from pyspark.sql.types import *
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel, Tokenizer, RegexTokenizer, StopWordsRemover
nltk.download("stopwords")
import pyLDAvis

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Paperspace\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
spark = SparkSession.builder.appName('NLP_topicModel').getOrCreate()


In [4]:
rawdata = spark.read.json("ARTICOLITV2.json", multiLine=True)
rawdata = rawdata.fillna({'testo_txt_it': ''})
rawdata = rawdata.withColumn("uid", monotonically_increasing_id())
rawdata = rawdata.withColumn("year_month", rawdata.published_dt.substr(1,7))
rawdata.show(3)


+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+---+----------+
|          _version_|                  id|language|language_s|        published_dt|        testo_txt_it|               url_s|uid|year_month|
+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+---+----------+
|1600612814188183552|2cfcaaac52cfbb66d...|        |   italian|2018-05-15T14:08:00Z|| 0
Dal 1° al 10 ...|http://omgili.com...|  0|   2018-05|
|1600612814388461568|0cf575e54190bff2e...|        |   italian|2018-05-15T14:08:00Z|15 maggio 2018 16...|http://omgili.com...|  1|   2018-05|
|1600612814394753024|b8dbc4e7826b41575...|        |   italian|2018-05-15T14:08:00Z|Leonardo Nappi 13...|http://omgili.com...|  2|   2018-05|
+-------------------+--------------------+--------+----------+--------------------+--------------------+--------------------+---+----------+
only showing 

# Puliamo i dati:
- Applichiamo la tokenization
- Rimuoviamo le stopwords. A questo insieme 

In [5]:
def myStemmer (record, array=False):
    stemmer = SnowballStemmer("italian")
    if array:
        text_out = [stemmer.stem(word) for word in record] 
    else:    
        text = record [2]#9
        text_out = [stemmer.stem(word) for word in text]
    return(text_out)

def removePunctuation(column):
    return trim(lower(regexp_replace(column, '[^A-Za-z]', ' '))).alias('sentence')

In [6]:
pulito = rawdata.select(removePunctuation(col("testo_txt_it")))
#Tokenize the text in the text column
tokenizer = Tokenizer(inputCol="sentence", outputCol="tokenized")#testo_txt_it
wordsDataFrame = tokenizer.transform(pulito)#rawdata

remover = StopWordsRemover(inputCol="tokenized", outputCol="tokenizedNew", stopWords = [" ", ""])
wordsDataFrame = remover.transform(wordsDataFrame)

udf_myStemmer = udf(myStemmer, (ArrayType(StringType()))) # if the function returns an int
wordsDataFrame = wordsDataFrame.withColumn("stemmed", udf_myStemmer(struct([wordsDataFrame[x] for x in wordsDataFrame.columns])))

# #remove 20 most occuring documents, documents with non numeric characters, and documents with <= 3 characters
cv_tmp = CountVectorizer(inputCol="stemmed", outputCol="tmp_vectors")
cv_tmp_model = cv_tmp.fit(wordsDataFrame)

stopWords = list(set(stopwords.words('english')))+list(set(stopwords.words('italian')))

top100 = list(cv_tmp_model.vocabulary[0:100])
stopWordsCustom = [" ","", "dal", "al","davan","avev","qualc", "qualcuno", "qualcosa", "avevano", "davanti", "aveva","e","avere", "fare","la","li", "lo", "gli", "essere", "solo", "per", "cosa", "ieri","disponibile", "anno", "detto", "quando","fatto", "sotto", "alcuna", "quali"]
#Add additional stopwords in th, is list

more_then_3_charachters = [word for word in cv_tmp_model.vocabulary if len(word) <= 3 ]
stopWords = list(set(stopwords.words('english')))+list(set(stopwords.words('italian')))
#Combine all the stopwords
stpw = StopWordsRemover.loadDefaultStopWords("italian") + stopWords+ stopWordsCustom +top100 + more_then_3_charachters 
stem_stopw = myStemmer(stpw, True) #stemming the stopwords

#Remove stopwords from the tokenized list
removerNew = StopWordsRemover(inputCol="stemmed", outputCol="final", stopWords =   stem_stopw+stpw) 
clean_text= removerNew.transform(wordsDataFrame)#dropping the stemmed stopwords from the stemmed word

In [7]:
clean_text.select("final").show(5, False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
train, test = clean_text.select("final").randomSplit([0.95, 0.05])
train.show(5)
test.show(5)

+--------------------+
|               final|
+--------------------+
|                  []|
|[accant, sardegn,...|
|[acciaier, tropp,...|
|[acqua, econom, b...|
|[adozion, distanz...|
+--------------------+
only showing top 5 rows

+--------------------+
|               final|
+--------------------+
|[appart, personag...|
|[appunt, iniz, do...|
|[aprit, cuor, ric...|
|[areol, favor, al...|
|[bologn, alleanz,...|
+--------------------+
only showing top 5 rows



In [15]:
cv_train = CountVectorizer(inputCol="final", outputCol="rawFeatures", vocabSize = 4000, minDF = 3, minTF = 2)
cvmodel = cv_train.fit(train)
cvDatasetTrain = cvmodel.transform(train)
cvDatasetTrain.select("final","rawFeatures").show(5)
cvDatasetTest = cvmodel.transform(test)

+--------------------+--------------------+
|               final|         rawFeatures|
+--------------------+--------------------+
|                  []|        (4000,[],[])|
|[accant, sardegn,...|(4000,[10,20,24,6...|
|[acciaier, tropp,...|(4000,[9,40,118,1...|
|[acqua, econom, b...|(4000,[1,7,10,19,...|
|[adozion, distanz...|(4000,[6,7,8,19,2...|
+--------------------+--------------------+
only showing top 5 rows



In [17]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(cvDatasetTrain)
rescaledDataTrain = idfModel.transform(cvDatasetTrain)
rescaledDataTest = idfModel.transform(cvDatasetTest)


In [18]:
k = 10 #numero topic
lda = LDA(k=k, seed=123, optimizer="online",featuresCol="features", subsamplingRate = 0.2, 
          learningDecay = 0.6, optimizeDocConcentration= True, maxIter = 120)


In [19]:
ldamodel = lda.fit(rescaledDataTrain)

In [20]:
prediction = ldamodel.transform(rescaledDataTest)
predictionTrain = ldamodel.transform(rescaledDataTrain)

In [21]:
def extract_data (ldamodel, countVectorizer_transf, countVectorizer_fit, df = train, results_pred = predictionTrain, column = "rawFeatures"):
    vocab = countVectorizer_fit.vocabulary
    rows_contents = countVectorizer_transf.select(column).take(df.count())
    doc_lengths=[]
    term_frequencies_list = []
    for i in range(len(rows_contents)):
        sparse_vector = rows_contents[i].asDict()[column] #è uno sparse vector se printato visualizza solo elem nonzero e
                                                    #rappresenta l'indice parola e la sua freq all'interno del documento
        doc_lengths.append(sparse_vector.numNonzeros()) #numero elementi non zero all'interno dell'array (senza le parentesi da 
                                                        # l'indice degli elem nonzero)
        term_frequencies_list.append(sparse_vector.toArray()) #toArray() mostra il vettore per intero, compresi gli zeri
    term_frequency = list(np.sum(term_frequencies_list, axis = 0)) #andiamo a sommare i vettori dei vari documenti, per ottenere le freq nel corpus
    to_drop = [i for i in range(len(doc_lengths)) if doc_lengths[i] == 0]       
    doc_lengths = [doc_lengths[i] for i in range(len(doc_lengths)) if i not in to_drop]   
    word_dists_topic = np.asmatrix(ldamodel.topicsMatrix().toArray())
    #qui abbiamo per ogni parola la distribuzione nei vari topic (1000, 10), noi vogliamo l'opposto (10, 1000)
    #quindi andiamo a trasporre la matrice appena creata e a trasformarla in una lista di liste (formato desiderato)
    topic_term_dists = word_dists_topic.transpose().tolist()
    pred_top_dists_pd = results_pred.select("topicDistribution").toPandas()
    doc_topic_dists = [list(pred_top_dists_pd.iloc[i][0]) for i in range(results_pred.count())]
    doc_topic_dists = [doc_topic_dists[i] for i in range(len(doc_topic_dists)) if i not in to_drop]       
#     results_visual = {"vocab": vocab, 'doc_lengths': doc_lengths , 'term_frequency': term_frequency, 'doc_topic_dists': doc_topic_dists, 'topic_term_dists': topic_term_dists}
    return(vocab, doc_lengths, term_frequency, topic_term_dists, doc_topic_dists)

In [24]:
vocab, doc_lengths, term_frequency, topic_term_dists, doc_topic_dists = extract_data (ldamodel, cvDatasetTrain, cvmodel, train, predictionTrain, "rawFeatures")


In [33]:
vocabTest, doc_lengthsTest, term_frequencyTest, topic_term_distsTest, doc_topic_distsTest = extract_data (ldamodel, cvDatasetTest, cvmodel, test, prediction, "rawFeatures")


In [34]:
prep_visualTest = pyLDAvis.prepare(topic_term_distsTest, doc_topic_distsTest, doc_lengthsTest, vocabTest, term_frequencyTest )

In [25]:
prep_visual = pyLDAvis.prepare(topic_term_dists, doc_topic_dists, doc_lengths, vocab, term_frequency )

C:\Users\Paperspace\Anaconda3\lib\site-packages\numexpr\cpuinfo.py:109: DeprecationWarning: invalid escape sequence \d
  nbits = re.compile('(\d+)bit').search(abits).group(1)
C:\Users\Paperspace\Anaconda3\lib\site-packages\numexpr\cpuinfo.py:662: DeprecationWarning: invalid escape sequence \s
  "\s+stepping\s+(?P<STP>\d+)", re.IGNORECASE)


In [26]:
pyLDAvis.display(prep_visual)

In [35]:
pyLDAvis.display(prep_visualTest)

In [ ]:
p = pyLDAvis.show(prep_visual)
pyLDAvis.save_html(p, 'lda.html')

In [27]:
print("Train-set results")
print("LogLikelihood", ldamodel.logLikelihood(rescaledDataTrain), "LogPerplexity", ldamodel.logPerplexity(rescaledDataTrain))

Train-set results
LogLikelihood -1860232.677848983 LogPerplexity 6.940232891372137


In [28]:
print("Test-set results")
print("LogLikelihood", ldamodel.logLikelihood(rescaledDataTest), "LogPerplexity", ldamodel.logPerplexity(rescaledDataTest))

Test-set results
LogLikelihood -142204.9969349004 LogPerplexity 10.617022446696343
